# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd

df = pd.read_csv('homepage_actions.csv')
df

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [3]:
df.duplicated().sum()

0

In [4]:
df.action.value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [5]:
df.group.value_counts()

control       4264
experiment    3924
Name: group, dtype: int64

In [8]:
# from solution

df['count'] = 1
control = df[df.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)



print("Sample sizes:\tControl: {}\tExperiment: {}".format(len(control), len(experiment)))
print("Total Clicks:\tControl: {}\tExperiment: {}".format(control.click.sum(), experiment.click.sum()))
print("Average click rate:\tControl: {}\tExperiment: {}".format(control.click.mean(), experiment.click.mean()))
control.head()

Sample sizes:	Control: 3332	Experiment: 2996
Total Clicks:	Control: 932.0	Experiment: 928.0
Average click rate:	Control: 0.2797118847539016	Experiment: 0.3097463284379172


action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0


In [37]:
from scipy import stats

result = stats.ttest_ind(control.click, experiment.click, equal_var = False)
p = result[1] / 2
p

0.004466402814337102

In [38]:
grouped = df.groupby('group')['action'].value_counts()
grouped

group       action
control     view      3332
            click      932
experiment  view      2996
            click      928
Name: action, dtype: int64

In [39]:
df['clicked'] = 0
df.loc[(df['action'] == 'click'), 'clicked'] = 1
print(df['clicked'].value_counts())



0    6328
1    1860
Name: clicked, dtype: int64


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [20]:
df['clicked']

0       0
1       0
2       0
3       0
4       0
       ..
8183    0
8184    0
8185    0
8186    0
8187    0
Name: clicked, Length: 8188, dtype: int64

In [40]:
from scipy import stats
filtered_dfcontrol = df[df['group'] == 'control']
filtered_dfexperiment = df[df['group'] == 'experiment']
filtered_dfcontrol

,timestamp,id,group,action,count,clicked
3,2016-09-24 19:59:02.646620,671993,control,view,1,0
9,2016-09-24 21:00:12.278374,560027,control,view,1,0
23,2016-09-25 00:25:14.141290,281985,control,view,1,0
24,2016-09-25 01:14:48.603202,407864,control,view,1,0
25,2016-09-25 02:16:11.046654,342984,control,view,1,0
...,...,...,...,...,...,...
8178,2017-01-18 08:17:12.675797,616692,control,view,1,0
8181,2017-01-18 09:07:37.661143,795585,control,view,1,0
8182,2017-01-18 09:09:17.363917,795585,control,click,1,1
8186,2017-01-18 10:08:51.588469,505451,control,view,1,0


In [41]:
ctrl_click = filtered_dfcontrol['clicked']
exp_click = filtered_dfexperiment['clicked']
ctrl_click

3       0
9       0
23      0
24      0
25      0
       ..
8178    0
8181    0
8182    1
8186    0
8187    0
Name: clicked, Length: 4264, dtype: int64

In [42]:



result2 = stats.ttest_ind(ctrl_click, exp_click, equal_var = False)
result2


Ttest_indResult(statistic=-1.9312441445072492, pvalue=0.05348777384439879)

In [43]:
p = result2[1] / 2
p

0.026743886922199395

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [44]:
#Your code here


control_rate = ctrl_click.mean()
expected_experiment_clicks_under_null = control_rate * len(exp_click)
print(expected_experiment_clicks_under_null)

857.6848030018762


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [45]:
#Your code here
import numpy as np


n = len(exp_click)
p = control_rate
vari = n * p * (1-p)
std = np.sqrt(vari)
print(std)

25.888551746650283


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [46]:
#Your code here
actual_experiment_clicks = exp_click.sum()
z_score = (actual_experiment_clicks - expected_experiment_clicks_under_null)/std
print(z_score)


2.71607302278784


In [47]:
import scipy.stats as stats
p_val = stats.norm.sf(z_score) #or 1 - stats.norm.cdf(z_score)
print(p_val)

0.003303067275926571


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: The z-score method resulted in a lower p-value. 

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.